# ARIMA forecasting 📈

Objectives:
* Walk-through the steps involved in time series forecasting
* Introduce ARIMA models
* Use an arima model to make a forecast
* Excercise: Automatize the analysis


In [4]:
# Import what you need here

#!pip install pmdarima and import it

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import pmdarima 

# Example 1 - airline passenger forecasts ✈️ 💺

Our goal here is to forecast the number of air-travel passengers (per month) over the next 12 months using historical data.

Get the airline passenger data




In [11]:
# CODE HERE
from pmdarima.datasets import load_airpassengers

START_DATE = '1949-01-01'
airline = load_airpassengers(as_series=True)

#there's no datetimeindex from the bundled dataset. So let's add one.
airline.index= pd.date_range(START_DATE, periods=len(airline), freq='MS')

Rather than a random train test split, when analysing time series data we must use the most recent data as the **testing set**.

TODO: Write a function which splits a dataset (a dataframe with time as index) and returns a train and test datasets, this function will have the split date as an argument.


In [ ]:
## Write your function and code here


In [ ]:
#### check train and test sizes are what we expect

## Explore TRAIN data

Like any prediction process we explore the data set to gain insights. But (as always) we only plot the TRAIN data! 

In [ ]:
# code here

Different months have different numbers of days. Soetimes we need to do what we can to make things easier for our model. Calcuating number of passengers per day might help remove some variation from the series...

In [ ]:
# code here

#### Explore subcomponents in series.

Before performing a forecast it is worth decomposing the time series into its components of trend, seasonality and noise.

* statsmodels has function called `seasonal_decompose()` for this task

In [ ]:
# code here

## Naive forecasting

We need a baseline (as always!) to know if our more complex models are any use! We explore two here:
* seasonal naive - a model which just takes the same period from the previous season. ( in this case the same month last year)
* naive - just carry forward the last value in the series

Given the strong seasonal component that was confirmed by the seasonal decomposition it may be a good idea to use a **seasonal naive** forecasting method.  This is part of the 'carry forward previous values' family of *naive* forecasting methods.  In general, if we have data with period $k$ are at time $t$ and we are predicting time $Y_{t+1}$ then we simply carry forward the value from $Y_{t+1-k}$. In other words we have yearly data so we just take the value from the same month last year.

In [ ]:
# code here

To see what this is doing lets plot its predictions for the whole TRAIN data set to which it has been fitted.

In [ ]:
# code here

Plotting the **residuals** can give us information about how the model is performing and the errors it is making.  Sometimes there are reffered to as **in sample** diagnostics. This just means we are looking at diagnostics of data which has been used to fit the model. 

In [ ]:
# code here

#### evaluating error

In the past we have done this. 

In [ ]:
# code here

For timeseries there are other approaches...

RMSE and MAE are called 'scale dependent' measures as the units and magnitude are specific to the problem and context.  An alternative approach is to use a scale invariant measure such as the **mean absolute percentage error (MAPE)**

The percentage error is given by $p_t = 100e_t/y_t$ where $e_t$ is the error in predicting $y_t$.  Therefore, MAPE = $mean(|p_t|)$. A limitation of MAPE is that it is inflated when the denominator is small relative to the absolute forecast error (such in the case of outliers or extreme unexpected events). It is also penalises negative errors more than positive errors.  A consequence of this property is that MAPE can lead to selecting a model that tends to under forecast.  The two following examples illustrate the issue. $$APE_{1} = \left| \frac{y_t - \hat{y_t}}{y_t} \right|= \left| \frac{150 - 100}{150} \right| = \frac{50}{150} = 33.33\%$$  

$$APE_{2} = \left| \frac{100 - 150}{100} \right| = \frac{50}{100} = 50\%$$

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    '''
    MAPE

    Parameters:
    --------
    y_true -- np.array actual observations from time series
    y_pred -- the predictions to evaluate

    Returns:
    -------
    float, scalar value representing the MAPE (0-100)
    '''
    # code here

## Task: Perform the same analysis for the naive prediction

* Have a go at using the `Naive1()` class.  It follows the same pattern as SNaive interface.  Instantiate a class.  call the `.fit(y_train)` method and then use the `.fittedvalues` and `.resid` properties for diagnostics.  
* Calculate the in-sample RMSE and MAPE
* What happens to the insample residuals if you fit the raw training data to the model?

In [ ]:
##### answer here

NOTE: The predictions we made above are only for the next step (month) in each timeseries! (not for the next 12 months) This means they are not correctly evaluating the error that we need for the specific task at hand. You can imagine for the simple naive model that a 12 month prediction actually looks like a flat line (using the most recent value). We will see this later.

## ARIMA

Let's try an ARIMA model at last. Using a classical statsitical approach you would select the order of model which produces a model with acceptable residual plots (remember the first part of linear regression module?). Selecting the best model can also be done automatically using packages which do it for you! Wohoo! 🎉

The `pmdarima` package is an excellent forecasting library for building ARIMA models.  I recommend it over and above the options available in core `statsmodels` package.  It is easier to use and offers an `auto_arima()` function that iteratively searches for a model that minimises the **Akaike Information Criterion (AIC)**

* ${\displaystyle \mathrm {AIC} \,=\,2k-2\ln({\hat {L}})}$

where $k$ = number of parameters in the model and $\hat{L}$ is the maximum value of the likelihood function for the model.  A likelihood function measures the 'goodness' of fit of a model to data given a set of parameters.  

This looks very complicated at first, but all we need to remember that the models we are working with are very flexible. This means that we can easily create complex models that overfit. Recall that overfitting is when a model will predict the training data exceptionally well, but will perform poorly on out of sample data.  The form of AIC means that it rewards models that fit the training data well, but also penalises models with high $k$ (complicated models with lots of parameters).  That means that AIC will prefer simpler models - in turn reducing overfitting.  That's a great formaula for automatically selecting a good ARIMA forecasting model.

There's a large amount of theory about how to build an ARIMA model.  But modern applications tend to opt for the auto approach.

In [ ]:
# code here

ARIMA models require data to be stationary. Stationarity includes that both the mean and variance do not change over time. ARIMA models can take into account an increasing mean over time, however not the variance. We therefore need another transformation. A log transformation will take care of this.

In [ ]:
# code here

#select a model that minimises AIC


In [ ]:
# code here

The best model selected is of order (2, 0, 0)x(0, 1, 1, 12) (and the residuals look acceptable). We will use these model in cross validation to estimate our model performance.

We can also specify a model with specific parameters as below.

In [ ]:
# code here

## Time series cross-validation

In reality we would use some sort of cross validation. However, we must be careful when using time series which approach we use. We must be careful not to give the model any information from the future that it would not otherwise have at the time of making a forecast. This means we cannot use the standard CV methods which randomise our data that we have used in the past.

In the classicial time series literature time series cross validation is called a **Rolling Forecasting Horizon**. This is explained in details [here](https://robjhyndman.com/hyndsight/tscv/)

## Timeseries CV with naive models on  airline data

In order to choose between our models we use CV to estimate how we think each will perform on new data.

### Naive models

In [ ]:
# code here

The prediction is better for the seasonal-naive forecast. We would choose this as our baseline.

### CV with ARIMA

In [ ]:
# code here

This is a far better model that either of the naive results. We would choose this model over the others. Is it suitable for use in practice?.... ca depend...

**TASK**
* go back and try different step values in the RollingForecastCV
* What changes?
* In each case what size is the data is the model training on?

# Predictions and evaluation on the TEST

In [ ]:
# code here

## Produce final forecast with chosen model

We want to predict the next 12 steps after the data we currently have.

In [ ]:
# code here

# Excercise 1 - Automatize the analysis

TODO: Write a class with the following methods:
* The necessary arguments to split the dataset, fit the auto_arima model etc.
* A `fit` method which fits the auto_arima naive and snaive models using a train dataset
* A `get_metrics` method which computes the MAPE for each models using a test dataset
* A `predict` method which takes as input a prediction horizon and returns predicted values.
* A `plot` method which plots the data, and if available the predictions 

Bonus: 
* Add a method get_cv Which performs cross validation
* To further automatize the process add an automatic detection of the period.
* Add the possibility to apply a preprocessing of the data before the fit: such as a log transformation.




In [ ]:
#### Write your code here

TODO: Test this method on all the other [11 datasets](https://alkaline-ml.com/pmdarima/modules/classes.html#module-pmdarima.datasets) of pmdarima. To help you, we provide the list of dataset names and a function to load any of these datasets.

On some of them the method does not work well, why? What could be done to improve it?

In [ ]:
#### Write your code here